In [ ]:
#!pip install tables
#!pip install pyarrow
#!pip install fastparquet

In [1]:
import os
import time
import zipfile
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

In [3]:
olist_list_csv_files=[
    'olist_sellers_dataset',
    'product_category_name_translation',
    'olist_products_dataset',
    'olist_order_reviews_dataset',
    'olist_orders_dataset',
    'olist_order_payments_dataset',
    'olist_order_items_dataset',
    'olist_geolocation_dataset',
    'olist_customers_dataset',
]

# File type .csv, .hdf, .parquet

In [ ]:
%%time
for file in olist_list_csv_files:
    df=pd.read_csv(f'../../data/olist/{file}.csv')
    df.to_hdf(f'../../data/olist/{file}.h5', key='data')
    df.to_parquet(f'../../data/olist/{file}.parquet')

In [31]:
%%time
file_name='olist_geolocation_dataset'
df_csv=pd.read_csv(f"../../data/olist/{file_name}.csv")

CPU times: total: 484 ms
Wall time: 475 ms


In [32]:
%time df_hdf=pd.read_hdf(f"../../data/olist/{file_name}.h5")

CPU times: total: 125 ms
Wall time: 124 ms


In [38]:
%time df_parquet=pd.read_parquet(f'../../data/olist/{file_name}.parquet')

CPU times: total: 203 ms
Wall time: 97 ms


In [36]:
df_parquet.shape

(1000163, 5)

In [37]:
%time df_parquet_columns=pd.read_parquet(f'../../data/olist/{file_name}.parquet', columns=["geolocation_zip_code_prefix", "geolocation_lat"])

CPU times: total: 46.9 ms
Wall time: 16 ms


# Get file list

In [4]:
def __get_list_files(path_root):
    path =path_root
    filelist = []
    filelist2 = []
    for root, dirs, files in os.walk(path):
        count_subdir=len(dirs)
        dic1={
                'root': root,
                'dirs': dirs,
                'file': None,
                'file_name': None,
                'file_ext': None,
                'file_size': None,
                'file_date_unix': None,
                'file_date': None,
                'count_subdir': count_subdir,
                'path_full': None,
        }
        filelist2.append(dic1)   
        for file in files:
            path_full=os.path.join(root,file)
            modTimesinceEpoc = os.path.getmtime(path_full)
            modificationTime = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(modTimesinceEpoc))
            dic1={
                'root': root,
                'dirs': dirs,
                'file': file,
                'file_name': os.path.splitext(file)[0],
                'file_ext': os.path.splitext(file)[1],
                'file_size': os.path.getsize(path_full),
                'file_date_unix': modTimesinceEpoc,
                'file_date': modificationTime,
                'count_subdir': count_subdir,
                'path_full': os.path.join(root,file)
            }
            filelist2.append(dic1)
            filelist.append(path_full)
    df_files=pd.DataFrame(filelist2)
    df_files['first_level']=df_files['root'].map(lambda x: '/'.join(x.split('/')[0:4]))
    df_files['file_size_mb']=df_files['file_size']/(1024*1024)
    df_files['second_level']=df_files['root'].map(lambda x: '/'.join(x.split('/')[0:5]))
    return df_files

In [5]:
path ="../../data/olist/"
df_files_all=__get_list_files(path)
df_files_all.head(2)

,root,dirs,file,file_name,file_ext,file_size,file_date_unix,file_date,count_subdir,path_full,first_level,file_size_mb,second_level
0,../../data/olist/,"[.ipynb_checkpoints, zip]",None,None,None,NaN,NaN,None,2,None,../../data/olist,NaN,../../data/olist/
1,../../data/olist/,"[.ipynb_checkpoints, zip]",olist_customers_dataset.csv,olist_customers_dataset,.csv,9033957.0,1.633105e+09,2021-10-01 19:08:42,2,../../data/olist/olist_customers_dataset.csv,../../data/olist,8.615453,../../data/olist/


In [6]:
df_files_all.groupby('file_ext')['file_size_mb'].sum()

file_ext
.csv        120.341296
.h5         163.692192
.parquet     54.455955
Name: file_size_mb, dtype: float64

# ZIP

In [15]:
zip_folder_name='../../data/zip'
unzip_folder_name='../../data/unzip'
try:
    os.mkdir(zip_folder_name)
except OSError as error: 
    print('Folder is exist')
try:
    os.mkdir(unzip_folder_name)
except OSError as error: 
    print('Folder is exist')

Folder is exist
Folder is exist


In [16]:
%%time
# One file
df_zip = zipfile.ZipFile(f'{zip_folder_name}/df_test.zip', 'w')
df_zip.write('../../data/olist/olist_order_reviews_dataset.h5', compress_type=zipfile.ZIP_DEFLATED)
df_zip.close()

CPU times: total: 1 s
Wall time: 997 ms


In [17]:
%%time
# few files

folder_zip = zipfile.ZipFile(f'{zip_folder_name}/df_test_folder.zip', 'w')
print(f'Size folder, mb: {df_files_all["file_size_mb"].sum()}')
fold='../../data/olist' 
for folder, subfolders, files in os.walk(fold):
    for file in files:
        folder_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), fold), 
                          compress_type = zipfile.ZIP_DEFLATED)
folder_zip.close()
df_files_zip=__get_list_files(zip_folder_name)
df_files_zip[['file','file_size_mb']]

Size folder, mb: 338.4894428253174
CPU times: total: 12.3 s
Wall time: 12.3 s


,file,file_size_mb
0,None,NaN
1,df_test.zip,13.030367
2,df_test_folder.zip,148.689166


In [18]:
%%time
# unzip
folder_zip = zipfile.ZipFile(f'{zip_folder_name}/df_test_folder.zip')
folder_zip.extractall(unzip_folder_name)
folder_zip.close()

CPU times: total: 1.2 s
Wall time: 1.23 s


In [21]:
# remove one file
file_path = f'{zip_folder_name}/df_test_folder.zip'
if os.path.exists(file_path):
    os.remove(file_path)
else:
    print("The system cannot find the file specified")

In [25]:
# delete all Files from a Directory
for file_name in os.listdir(zip_folder_name):
    file = f'{zip_folder_name}/{file_name}'
    if os.path.isfile(file):
        print('Deleting file:', file)
        os.remove(file)

Deleting file: ../../data/zip/df_test.zip


In [27]:
# remove clear folder
os.rmdir(zip_folder_name)

In [28]:
# Deleting an non-empty folder
import shutil
shutil.rmtree(unzip_folder_name, ignore_errors=True)
print("Deleted '%s' directory successfully" % unzip_folder_name)

Deleted '../../data/unzip' directory successfully
